# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 9:34AM,241279,10,0085840539,ISDIN Corporation,Released
1,Jul 7 2022 9:34AM,241279,10,0085840557,ISDIN Corporation,Released
2,Jul 7 2022 9:34AM,241279,10,0085840572,ISDIN Corporation,Released
3,Jul 7 2022 9:34AM,241279,10,0085840574,ISDIN Corporation,Released
4,Jul 7 2022 9:34AM,241279,10,0085840573,ISDIN Corporation,Released
5,Jul 7 2022 9:34AM,241279,10,0085840448,ISDIN Corporation,Released
6,Jul 7 2022 9:34AM,241279,10,0085840456,ISDIN Corporation,Released
7,Jul 7 2022 9:34AM,241279,10,0085840484,ISDIN Corporation,Released
8,Jul 7 2022 9:34AM,241279,10,0085840457,ISDIN Corporation,Released
9,Jul 7 2022 9:34AM,241279,10,0085842212,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,241273,Released,16
49,241274,Released,16
50,241275,Released,1
51,241278,Released,7
52,241279,Released,21


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241273,NaN,NaN,16.0
241274,NaN,NaN,16.0
241275,NaN,NaN,1.0
241278,NaN,NaN,7.0
241279,NaN,NaN,21.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241180,0.0,0.0,1.0
241183,0.0,1.0,0.0
241184,0.0,0.0,14.0
241186,0.0,0.0,9.0
241187,0.0,0.0,57.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241180,0,0,1
241183,0,1,0
241184,0,0,14
241186,0,0,9
241187,0,0,57


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241180,0,0,1
1,241183,0,1,0
2,241184,0,0,14
3,241186,0,0,9
4,241187,0,0,57


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241180,,,1
1,241183,,1,
2,241184,,,14
3,241186,,,9
4,241187,,,57


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation
21,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC"
28,Jul 7 2022 9:22AM,241275,12,Hush Hush
29,Jul 7 2022 9:22AM,241274,12,Hush Hush
45,Jul 7 2022 9:21AM,241273,10,ISDIN Corporation
61,Jul 7 2022 8:52AM,241268,10,Bio-PRF
75,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation
109,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation
147,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation
185,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,,,21
1,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",,,7
2,Jul 7 2022 9:22AM,241275,12,Hush Hush,,,1
3,Jul 7 2022 9:22AM,241274,12,Hush Hush,,,16
4,Jul 7 2022 9:21AM,241273,10,ISDIN Corporation,,,16
5,Jul 7 2022 8:52AM,241268,10,Bio-PRF,,,14
6,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,,,34
7,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,,,38
8,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,,,38
9,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,21,,
1,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",7,,
2,Jul 7 2022 9:22AM,241275,12,Hush Hush,1,,
3,Jul 7 2022 9:22AM,241274,12,Hush Hush,16,,
4,Jul 7 2022 9:21AM,241273,10,ISDIN Corporation,16,,
5,Jul 7 2022 8:52AM,241268,10,Bio-PRF,14,,
6,Jul 7 2022 8:44AM,241269,10,ISDIN Corporation,34,,
7,Jul 7 2022 8:41AM,241266,10,ISDIN Corporation,38,,
8,Jul 7 2022 8:41AM,241265,10,ISDIN Corporation,38,,
9,Jul 7 2022 8:38AM,241267,10,ISDIN Corporation,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,21,,
1,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",7,,
2,Jul 7 2022 9:22AM,241275,12,Hush Hush,1,,
3,Jul 7 2022 9:22AM,241274,12,Hush Hush,16,,
4,Jul 7 2022 9:21AM,241273,10,ISDIN Corporation,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,21.0,NaN,NaN
1,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",7.0,NaN,NaN
2,Jul 7 2022 9:22AM,241275,12,Hush Hush,1.0,NaN,NaN
3,Jul 7 2022 9:22AM,241274,12,Hush Hush,16.0,NaN,NaN
4,Jul 7 2022 9:21AM,241273,10,ISDIN Corporation,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,21.0,0.0,0.0
1,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",7.0,0.0,0.0
2,Jul 7 2022 9:22AM,241275,12,Hush Hush,1.0,0.0,0.0
3,Jul 7 2022 9:22AM,241274,12,Hush Hush,16.0,0.0,0.0
4,Jul 7 2022 9:21AM,241273,10,ISDIN Corporation,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7237054,556.0,1.0,0.0
12,723757,18.0,0.0,0.0
16,723709,1.0,2.0,0.0
19,723704,64.0,2.0,8.0
20,482418,28.0,1.0,0.0
21,1929920,7.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7237054,556.0,1.0,0.0
1,12,723757,18.0,0.0,0.0
2,16,723709,1.0,2.0,0.0
3,19,723704,64.0,2.0,8.0
4,20,482418,28.0,1.0,0.0
5,21,1929920,7.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,556.0,1.0,0.0
1,12,18.0,0.0,0.0
2,16,1.0,2.0,0.0
3,19,64.0,2.0,8.0
4,20,28.0,1.0,0.0
5,21,7.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,556.0
1,12,Released,18.0
2,16,Released,1.0
3,19,Released,64.0
4,20,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,8.0,0.0,0.0
Executing,1.0,0.0,2.0,2.0,1.0,1.0
Released,556.0,18.0,1.0,64.0,28.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,1.0,0.0,2.0,2.0,1.0,1.0
2,Released,556.0,18.0,1.0,64.0,28.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,1.0,0.0,2.0,2.0,1.0,1.0
2,Released,556.0,18.0,1.0,64.0,28.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()